In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [30]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# rankings = pd.read_excel(file_name)

In [5]:
single_games = all_df.sort_values('fantasyPoints', ascending=False)

goaltenders_single = single_games.loc[single_games['position'] == 'G']
skaters_single = single_games.loc[single_games['position'] != 'G']
defense_single = skaters_single.loc[skaters_single['position'] == 'D']
forwards_single = skaters_single.loc[skaters_single['position'] != 'D']

In [8]:
import pymannkendall as mk

def convert_to_minutes_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return f"{minutes}:{remaining_seconds:02d}"


test_data = defense_single.loc[defense_single['name'] == 'Tony DeAngelo'].sort_values('gameDate', ascending=True)
test_data = test_data['toi'].to_list()
result = mk.original_test(test_data)
print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
print(', '.join(formatted_times))

ZeroDivisionError: float division by zero

In [9]:
d_cull = defense_single['name'].value_counts()
d_cull = d_cull[d_cull == 1]
d_cull = d_cull.index.to_list()
d_list = defense_single['name'].unique()
d_list = d_list.tolist()
d_list = [name for name in d_list if name not in d_cull]

In [13]:

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        print()

print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        print()
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        print()
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))
        print()

B. Byram's ice time is increasing and the Tau score is 0.29
20:29, 18:57, 17:24, 17:37, 14:03, 19:46, 21:22, 19:39, 18:43, 21:08, 18:28, 18:43, 21:07, 23:01, 23:32, 20:14, 23:13, 22:46, 20:35, 19:19, 19:40, 21:58, 25:15, 22:04, 14:52, 20:46, 20:22, 22:21

O. Ekman-Larsson's ice time is decreasing and the Tau score is -0.61
26:32, 26:16, 25:19, 23:56, 22:04, 22:37, 23:53, 20:22, 26:24, 23:56, 22:21, 23:53, 21:10, 20:37, 23:48, 28:22, 17:59, 18:50, 16:13, 17:47, 13:02, 14:15, 17:13, 17:18, 15:04, 16:16, 17:58

A. Romanov's ice time is increasing and the Tau score is 0.4
22:04, 17:48, 22:06, 21:52, 17:09, 20:35, 23:08, 21:35, 25:16, 25:23, 18:54, 23:37, 21:51, 23:07, 19:33, 18:40, 22:52, 22:04, 30:02, 22:52, 23:43, 24:30, 25:44, 24:41, 25:59, 23:24, 24:17

A. Carrier's ice time is decreasing and the Tau score is -0.44
20:39, 20:50, 19:22, 18:05, 22:37, 21:49, 20:06, 18:38, 19:34, 17:33, 19:20, 20:20, 17:39, 17:33, 15:32, 17:32, 20:53, 19:24, 18:16, 16:24, 15:11, 1:39, 16:00, 20:16

G. For

A. Pelech's fantasy points are decreasing and the Tau score is -0.4

Z. Whitecloud's fantasy points are increasing and the Tau score is 0.47

P. Mintyukov's fantasy points are decreasing and the Tau score is -0.28

D. Kulikov's fantasy points are decreasing and the Tau score is -0.29

J. McCabe's fantasy points are increasing and the Tau score is 0.37

B. Pesce's fantasy points are decreasing and the Tau score is -0.47

E. Zamula's fantasy points are decreasing and the Tau score is -0.33

J. Klingberg's fantasy points are decreasing and the Tau score is -0.52

M. Staal's fantasy points are decreasing and the Tau score is -0.49

S. Bolduc's fantasy points are increasing and the Tau score is 0.49

Z. Bogosian's fantasy points are increasing and the Tau score is 0.44

--------------------
J. Trouba's power-play ice time is increasing and the Tau score is 0.28
0:00, 0:06, 0:00, 0:01, 0:00, 0:00, 0:05, 0:29, 0:00, 0:07, 0:16, 0:00, 0:00, 0:06, 0:00, 0:50, 1:13, 0:14, 0:01, 0:20, 0:28, 0:00,

M. Staal's shots are decreasing and the Tau score is -0.44
1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0



In [14]:
f_cull = forwards_single['name'].value_counts()
f_cull = f_cull[f_cull == 1]
f_cull = f_cull.index.to_list()
f_list = forwards_single['name'].unique()
f_list = f_list.tolist()
f_list = [name for name in f_list if name not in f_cull]

In [15]:
#Print a list of forward trends
    
for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        print()

print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num:.1f}' for num in test_data]
        print(', '.join(formatted_numbers))
        print()
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        print()
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))
        print()

C. Coyle's ice time is increasing and the Tau score is 0.36
15:56, 17:44, 16:39, 17:01, 15:28, 15:12, 19:32, 14:13, 19:22, 19:30, 15:44, 17:29, 17:18, 17:09, 18:35, 18:05, 18:45, 17:30, 17:56, 15:42, 17:55, 17:16, 19:45, 18:48, 20:03, 20:17

B. Jenner's ice time is decreasing and the Tau score is -0.27
21:01, 17:48, 19:09, 20:24, 22:46, 25:20, 22:07, 22:25, 19:10, 18:19, 18:08, 19:55, 17:27, 17:52, 17:45, 20:43, 17:51, 20:16, 19:36, 15:30, 19:15, 17:40, 19:37, 17:38, 23:24, 20:01, 17:56, 18:14, 12:16

R. Hartman's ice time is decreasing and the Tau score is -0.33
17:56, 17:32, 18:38, 16:27, 16:23, 17:28, 17:28, 17:40, 21:38, 16:05, 16:25, 15:21, 14:46, 15:22, 13:02, 17:54, 18:14, 13:13, 14:02, 12:43, 12:42, 18:51, 16:42

P. Buchnevich's ice time is increasing and the Tau score is 0.46
21:21, 5:27, 17:25, 18:42, 16:45, 20:16, 17:40, 18:14, 17:31, 22:02, 18:30, 16:49, 20:56, 19:38, 22:50, 20:12, 18:38, 18:06, 23:28, 20:21, 19:54, 20:58, 24:03, 23:28, 23:40

S. Reinhart's ice time is decr

S. Laughton's ice time is increasing and the Tau score is 0.37
14:22, 13:38, 14:45, 15:29, 16:36, 13:13, 16:28, 15:08, 10:28, 11:33, 16:49, 15:42, 17:57, 17:33, 16:09, 15:38, 14:16, 16:51, 10:40, 17:33, 12:45, 12:25, 19:10, 19:10, 19:52, 20:40, 22:09

O. Palat's ice time is increasing and the Tau score is 0.29
14:35, 14:17, 15:55, 16:16, 13:54, 13:37, 12:07, 14:25, 16:05, 15:21, 15:38, 15:21, 16:31, 15:33, 18:22, 22:09, 16:07, 16:54, 13:13, 15:51, 13:02, 17:22, 17:41, 16:34, 15:54, 16:04

J. Anderson's ice time is decreasing and the Tau score is -0.3
17:31, 13:39, 15:13, 18:27, 15:10, 18:05, 17:44, 17:09, 17:21, 15:59, 19:12, 18:44, 16:37, 20:29, 16:49, 18:24, 16:44, 14:11, 16:18, 15:13, 13:41, 15:15, 13:27, 13:46, 16:13, 13:27, 14:24, 12:27, 14:00, 13:56, 16:46, 14:02, 14:36, 15:27, 16:58, 15:28, 16:21, 12:40

F. Gaudreau's ice time is decreasing and the Tau score is -0.4
15:36, 9:51, 18:02, 15:13, 15:14, 15:23, 15:04, 14:43, 8:38, 13:43, 13:49, 12:27, 12:00, 13:48, 13:31, 13:54

B. L

B. McGinn's fantasy points are decreasing and the Tau score is -0.56
1.1, 1.0, 0.9, 0.9, 0.3, 2.2, 0.9, 0.6, 0.6, 0.0

T. Motte's fantasy points are decreasing and the Tau score is -0.4
0.7, 1.1, 2.2, 0.7, 0.2, 0.1, 0.4, 0.6, 1.4, 0.5, 0.9, 0.1, 0.2, 0.0, 0.3, 0.2, 0.0, 0.6, 0.1

N. Bastian's fantasy points are decreasing and the Tau score is -0.32
1.1, 0.7, 1.1, 0.7, 0.7, 0.8, 0.5, 1.8, 0.4, 0.3, 0.7, 0.6, 0.2, 2.1, 0.4, 0.1, 0.2, 0.9, 0.1, 0.6, 0.1, 1.4, 0.3, 0.6

A. Jonsson-Fjallby's fantasy points are increasing and the Tau score is 0.67
0.3, 0.1, 0.1, 0.1, 0.3, 0.6, 0.4, 0.8, 1.5

N. Alexandrov's fantasy points are decreasing and the Tau score is -0.51
0.9, 0.6, 0.7, 0.2, 0.2, 0.1, 0.6, 0.2, 0.4, 0.0

--------------------
C. Coyle's power-play ice time is increasing and the Tau score is 0.36
0:03, 0:29, 0:00, 0:00, 0:00, 0:00, 1:42, 0:00, 0:59, 1:16, 2:27, 1:48, 1:21, 0:22, 2:59, 1:34, 0:49, 2:28, 1:59, 1:18, 1:43, 2:11, 1:03, 4:03, 0:00, 1:24

K. Connor's power-play ice time is d

A. Texier's power-play ice time is decreasing and the Tau score is -0.43
1:25, 1:03, 4:21, 2:38, 0:00, 0:53, 0:00, 0:05, 0:19, 0:13, 0:00, 1:20, 0:44, 0:40, 0:00, 0:00, 0:10, 0:00, 1:02, 0:00, 0:00, 0:00, 0:00, 0:04, 0:00, 0:00, 0:00, 0:00

R. Donato's power-play ice time is decreasing and the Tau score is -0.35
4:34, 2:53, 8:12, 3:53, 4:01, 5:47, 1:33, 0:00, 0:55, 0:00, 0:00, 0:12, 1:48, 0:29, 0:00, 0:00, 3:29, 1:46, 2:08, 1:21, 1:02, 0:38, 1:07, 0:00, 0:00

D. Dube's power-play ice time is decreasing and the Tau score is -0.53
1:48, 1:18, 3:42, 2:37, 1:16, 0:43, 2:30, 0:39, 3:18, 0:00, 0:00, 0:03, 0:18, 0:06, 0:00, 0:18, 0:00, 0:09, 0:00, 0:00, 0:06, 0:00, 0:00, 0:00, 0:09, 0:00, 0:00

J. Drouin's power-play ice time is increasing and the Tau score is 0.52
0:23, 1:16, 1:19, 1:11, 2:11, 0:21, 2:50, 3:44, 0:19, 0:31, 1:59, 0:58, 2:17, 1:52, 1:04, 2:23, 5:57, 3:15, 7:20, 4:37, 5:28, 2:41, 4:46, 6:01, 3:06, 5:22

M. Johansson's power-play ice time is decreasing and the Tau score is -0.34

In [31]:
all_df

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,team,opponent,secondaryPosition,tertiaryPosition,gameDate,gameTime,gameId,gamesPlayed,fantasyPoints,specialTeams
0,8478178,43,D. Raddysh,D,0,0,0,1,0,1,...,TBL,NSH,D,D,2023-10-10,Tuesday 05:30 PM,2023020001,1,0.9,0
1,8475177,44,C. de Haan,D,0,0,0,0,0,2,...,TBL,NSH,D,D,2023-10-10,Tuesday 05:30 PM,2023020001,1,0.7,0
2,8480246,48,N. Perbix,D,0,0,0,-1,2,0,...,TBL,NSH,D,D,2023-10-10,Tuesday 05:30 PM,2023020001,1,0.6,0
3,8475167,77,V. Hedman,D,0,1,1,-1,0,0,...,TBL,NSH,D,D,2023-10-10,Tuesday 05:30 PM,2023020001,1,1.9,0
4,8478416,81,E. Cernak,D,0,0,0,0,0,6,...,TBL,NSH,D,D,2023-10-10,Tuesday 05:30 PM,2023020001,1,1.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15505,8482124,55,Q. Byfield,R,0,0,0,0,2,0,...,LAK,NYI,W,F,2023-12-09 00:00:00,Saturday 07:30 PM,2023020415,1,0.3,0
15506,8473453,61,T. Lewis,C,0,0,0,0,0,2,...,LAK,NYI,C,F,2023-12-09 00:00:00,Saturday 07:30 PM,2023020415,1,0.8,0
15507,8482155,78,A. Laferriere,R,0,0,0,0,2,1,...,LAK,NYI,W,F,2023-12-09 00:00:00,Saturday 07:30 PM,2023020415,1,1.5,0
15508,8479400,80,P. Dubois,C,0,0,0,0,0,1,...,LAK,NYI,C,F,2023-12-09 00:00:00,Saturday 07:30 PM,2023020415,1,0.7,0


In [33]:
new_all_df = pd.DataFrame()

for index, row in homeOnlySked.iterrows():
    temp_home = all_df.loc[(all_df['gameId'] == row['gameID']) & (all_df['team'] == row['homeTeam'])]
    temp_away = all_df.loc[(all_df['gameId'] == row['gameID']) & (all_df['team'] == row['awayTeam'])]
    
    temp_home['pp_rank'] = temp_home['powerPlayToi'].rank(ascending=False)
    temp_away['pp_rank'] = temp_away['powerPlayToi'].rank(ascending=False)
    
    new_all_df = pd.concat([new_all_df, temp_home, temp_away])


new_all_df

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,opponent,secondaryPosition,tertiaryPosition,gameDate,gameTime,gameId,gamesPlayed,fantasyPoints,specialTeams,pp_rank
36,8475188,3,B. McNabb,D,0,1,1,3,0,3,...,SEA,D,D,2023-10-10,Tuesday 10:30 PM,2023020003,1,2.8,0,16.0
37,8474565,7,A. Pietrangelo,D,0,0,0,0,0,0,...,SEA,D,D,2023-10-10,Tuesday 10:30 PM,2023020003,1,1.3,0,8.0
38,8479980,14,N. Hague,D,0,0,0,0,0,1,...,SEA,D,D,2023-10-10,Tuesday 10:30 PM,2023020003,1,1.2,0,12.0
39,8477018,17,B. Hutton,D,0,0,0,1,0,0,...,SEA,D,D,2023-10-10,Tuesday 10:30 PM,2023020003,1,1.1,0,9.0
40,8477447,27,S. Theodore,D,0,0,0,2,0,2,...,SEA,D,D,2023-10-10,Tuesday 10:30 PM,2023020003,1,1.2,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11725,8477511,39,A. Mantha,R,0,0,0,1,0,1,...,SJS,W,F,2023-11-27 00:00:00,Monday 10:30 PM,2023020326,1,0.8,0,5.0
11726,8476880,43,T. Wilson,R,0,0,0,0,0,4,...,SJS,W,F,2023-11-27 00:00:00,Monday 10:30 PM,2023020326,1,1.0,0,3.5
11727,8479547,45,M. Phillips,R,0,0,0,-1,0,0,...,SJS,W,F,2023-11-27 00:00:00,Monday 10:30 PM,2023020326,1,0.3,0,15.0
11728,8479359,47,B. Malenstyn,L,0,0,0,1,0,3,...,SJS,W,F,2023-11-27 00:00:00,Monday 10:30 PM,2023020326,1,1.4,0,15.0


In [34]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first',
        'pp_rank': 'sum'
    }).reset_index()


    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [44]:
sum_stats = summary_statistics(new_all_df).sort_values('pp_rank', ascending=True)
sum_stats['pp_actual'] = sum_stats['pp_rank'] / sum_stats['gamesPlayed']
sum_stats = sum_stats.sort_values('pp_actual', ascending=True)

focus = sum_stats.loc[(sum_stats['FPPG'] <= 1.7) & (sum_stats['pp_actual'] <= 5)]
focus

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,pp_rank,FPP60,FPPG,pp_actual
6,8471214,A. Ovechkin,WSH,L,29145,25,5,11,5,90,...,7380,0,-2,40.7,W,F,38.5,5.03,1.63,1.540000
319,8477986,B. Montour,FLA,D,15604,11,0,2,1,26,...,2293,263,3,10.5,D,D,20.0,2.42,0.95,1.818182
628,8481523,K. Dach,MTL,C,1562,2,0,2,0,3,...,411,15,2,2.6,C,F,4.5,5.99,1.30,2.250000
41,8474141,P. Kane,DET,R,3430,3,1,0,1,11,...,575,0,-3,4.6,W,F,7.0,4.83,1.53,2.333333
3,8470621,C. Perry,CHI,R,14067,16,4,5,2,26,...,3706,0,-5,21.7,W,F,44.5,5.55,1.36,2.781250
731,8482684,L. Hughes,NJD,D,31264,26,4,12,8,40,...,4942,20,-1,35.1,D,D,78.0,4.04,1.35,3.000000
307,8477950,T. DeAngelo,CAR,D,15535,16,1,6,4,28,...,3273,67,-10,17.4,D,D,54.0,4.03,1.09,3.375000
354,8478401,P. Zacha,BOS,C,28647,26,8,11,6,47,...,5141,3123,2,43.1,C,F,90.0,5.42,1.66,3.461538
636,8481540,C. Caufield,MTL,R,31674,28,7,13,9,105,...,6709,0,0,45.1,W,F,97.5,5.13,1.61,3.482143
248,8477426,N. Paul,TBL,L,30091,29,9,3,6,57,...,6454,1199,-16,41.3,W,F,105.0,4.94,1.42,3.620690


In [40]:
compare = sum_stats.loc[sum_stats['pp_actual'] <= 5]
compare

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,pp_rank,FPP60,FPPG,pp_actual
6,8471214,A. Ovechkin,WSH,L,29145,25,5,11,5,90,...,7380,0,-2,40.7,W,F,38.5,5.03,1.63,1.540000
159,8476453,N. Kucherov,TBL,R,36222,28,19,28,22,137,...,6992,22,-1,97.7,W,F,45.0,9.71,3.49,1.607143
57,8474600,R. Josi,NSH,D,41567,28,6,16,11,102,...,7391,1092,-3,70.5,D,D,49.0,6.11,2.52,1.750000
319,8477986,B. Montour,FLA,D,15604,11,0,2,1,26,...,2293,263,3,10.5,D,D,20.0,2.42,0.95,1.818182
517,8480036,M. Heiskanen,DAL,D,40878,27,3,16,9,56,...,4949,2008,-6,56.2,D,D,49.5,4.95,2.08,1.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,8475793,R. Johansen,COL,C,24181,28,9,2,5,46,...,5237,341,-4,30.5,C,F,138.0,4.54,1.09,4.928571
730,8482679,M. Coronato,CGY,R,9146,11,1,1,1,24,...,1946,0,-9,6.0,W,F,54.5,2.36,0.55,4.954545
499,8479999,C. Mittelstadt,BUF,C,33248,29,6,16,3,46,...,4642,373,1,42.2,C,F,144.5,4.57,1.46,4.982759
749,8483490,P. Mintyukov,ANA,D,30716,27,1,12,4,45,...,4525,798,-8,38.3,D,D,135.0,4.49,1.42,5.000000


In [45]:
for index, row in focus.iterrows():
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['pp_actual'], 2)} APPTR; {round(row['FPPG'], 2)} FPPG): "
)

<p><b>Alex Ovechkin, L, Washington Capitals </b>(1.54 APPTR; 1.63 FPPG): 
<p><b>Brandon Montour, D, Florida Panthers </b>(1.82 APPTR; 0.95 FPPG): 
<p><b>Kirby Dach, C, Montréal Canadiens </b>(2.25 APPTR; 1.3 FPPG): 
<p><b>Patrick Kane, R, Detroit Red Wings </b>(2.33 APPTR; 1.53 FPPG): 
<p><b>Corey Perry, R, Chicago Blackhawks </b>(2.78 APPTR; 1.36 FPPG): 
<p><b>Luke Hughes, D, New Jersey Devils </b>(3.0 APPTR; 1.35 FPPG): 
<p><b>Tony DeAngelo, D, Carolina Hurricanes </b>(3.38 APPTR; 1.09 FPPG): 
<p><b>Pavel Zacha, C, Boston Bruins </b>(3.46 APPTR; 1.66 FPPG): 
<p><b>Cole Caufield, R, Montréal Canadiens </b>(3.48 APPTR; 1.61 FPPG): 
<p><b>Nicholas Paul, L, Tampa Bay Lightning </b>(3.62 APPTR; 1.42 FPPG): 
<p><b>Trevor Zegras, C, Anaheim Ducks </b>(3.62 APPTR; 0.94 FPPG): 
<p><b>John Klingberg, D, Toronto Maple Leafs </b>(3.64 APPTR; 1.11 FPPG): 
<p><b>Martin Necas, C, Carolina Hurricanes </b>(3.65 APPTR; 1.56 FPPG): 
<p><b>Johnny Gaudreau, L, Columbus Blue Jackets </b>(3.7 APPTR; 1.1 FP

In [48]:
sum_stats.loc[sum_stats['team'] == 'SJS']

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,pp_rank,FPP60,FPPG,pp_actual
130,8475798,M. Granlund,SJS,C,26221,21,3,15,5,43,...,3897,2691,0,37.6,C,F,65.0,5.16,1.79,3.095238
196,8476881,T. Hertl,SJS,C,34384,27,9,15,7,68,...,5212,2624,-14,58.2,C,F,91.5,6.09,2.16,3.388889
594,8480884,C. Addison,SJS,D,30300,28,1,10,6,39,...,5600,10,-12,32.3,D,D,108.5,3.84,1.15,3.875000
715,8482222,A. Barabanov,SJS,L,10695,10,1,3,0,14,...,1654,0,-3,8.7,W,F,43.5,2.93,0.87,4.350000
728,8482667,W. Eklund,SJS,L,29721,27,6,7,5,41,...,4404,792,-8,38.8,W,F,123.5,4.70,1.44,4.574074
68,8474884,M. Hoffman,SJS,C,23161,27,8,3,3,32,...,3010,6,-8,32.2,C,F,169.5,5.00,1.19,6.277778
536,8480188,F. Zetterlund,SJS,L,29730,28,8,4,5,67,...,3500,736,-7,46.0,W,F,183.0,5.57,1.64,6.535714
244,8477407,A. Duclair,SJS,L,24635,25,6,4,3,43,...,3070,327,-6,30.1,W,F,172.0,4.40,1.20,6.880000
339,8478099,K. Labanc,SJS,R,15292,20,1,4,2,29,...,1489,0,-11,15.2,W,F,154.5,3.58,0.76,7.725000
465,8479439,J. MacDonald,SJS,D,9647,13,4,1,3,23,...,1122,0,-9,16.5,D,D,102.5,6.16,1.27,7.884615


In [49]:
focus

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,pp_rank,FPP60,FPPG,pp_actual
6,8471214,A. Ovechkin,WSH,L,29145,25,5,11,5,90,...,7380,0,-2,40.7,W,F,38.5,5.03,1.63,1.540000
319,8477986,B. Montour,FLA,D,15604,11,0,2,1,26,...,2293,263,3,10.5,D,D,20.0,2.42,0.95,1.818182
628,8481523,K. Dach,MTL,C,1562,2,0,2,0,3,...,411,15,2,2.6,C,F,4.5,5.99,1.30,2.250000
41,8474141,P. Kane,DET,R,3430,3,1,0,1,11,...,575,0,-3,4.6,W,F,7.0,4.83,1.53,2.333333
3,8470621,C. Perry,CHI,R,14067,16,4,5,2,26,...,3706,0,-5,21.7,W,F,44.5,5.55,1.36,2.781250
731,8482684,L. Hughes,NJD,D,31264,26,4,12,8,40,...,4942,20,-1,35.1,D,D,78.0,4.04,1.35,3.000000
307,8477950,T. DeAngelo,CAR,D,15535,16,1,6,4,28,...,3273,67,-10,17.4,D,D,54.0,4.03,1.09,3.375000
354,8478401,P. Zacha,BOS,C,28647,26,8,11,6,47,...,5141,3123,2,43.1,C,F,90.0,5.42,1.66,3.461538
636,8481540,C. Caufield,MTL,R,31674,28,7,13,9,105,...,6709,0,0,45.1,W,F,97.5,5.13,1.61,3.482143
248,8477426,N. Paul,TBL,L,30091,29,9,3,6,57,...,6454,1199,-16,41.3,W,F,105.0,4.94,1.42,3.620690
